In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle

df = pd.read_csv('tweet_emotions.csv')

analyzer = SentimentIntensityAnalyzer()

def map_intensity_to_emotion(score):
    if score >= 0.5:
        return 'joy'
    elif score >= 0.05 and score < 0.5:
        return 'pleasant'
    elif score >= -0.05 and score < 0.05:
        return 'not bad'
    elif score > -0.5 and score < -0.05:
        return 'disappointed'
    else:
        return 'sad'

df['vader_score'] = df['content'].apply(lambda text: analyzer.polarity_scores(text)['compound'])

df['emotion'] = df['vader_score'].apply(map_intensity_to_emotion)

df['content'] = df['content'].str.lower()

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(df['content'])
y = df['emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

with open('senti_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

clf = pickle.load(open('senti_model.pkl', 'rb'))

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

disappointed       0.56      0.45      0.50      1204
         joy       0.72      0.81      0.76      2264
     not bad       0.77      0.92      0.84      2049
    pleasant       0.59      0.50      0.54      1608
         sad       0.65      0.50      0.57       875

    accuracy                           0.69      8000
   macro avg       0.66      0.64      0.64      8000
weighted avg       0.68      0.69      0.68      8000



In [3]:
text_input = "this product is not bad"
text_input = text_input.lower()

text_feature_vector = vectorizer.transform([text_input])

predicted_emotion = clf.predict(text_feature_vector)

print("Predicted Emotion:", predicted_emotion)


Predicted Emotion: ['sad']
